### Searching for sequence

In [5]:
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast('blastn', 'nt', '8332116')

In [8]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)  # or use a for loop

ValueError: Your XML file was empty

### Bio.SearchIO object model - representation of your search results

In [9]:
from Bio import SearchIO
blast_qresult = SearchIO.read('my_blast.xml', 'blast-xml')
print(blast_qresult)

Program: blastn (2.13.0+)
  Query: [Glycine (94336)
         max] Forrest B100B10 BAC sequence, Rhg4
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      4  gi|1003312939|gb|KT873795.1|  Glycine max cultivar Forr...
            1     11  gi|148372212|gb|EF623856.1|  Glycine max cultivar Willi...
            2      1  gi|148372211|gb|EF623855.1|  Glycine max cultivar Willi...
            3      1  gi|308083482|gb|HM367084.1|  Glycine max cultivar Forre...
            4      2  gi|21239383|gb|AF506518.1|  Glycine max receptor-like k...
            5      1  gi|226693208|dbj|AB495281.1|  Glycine max Rhg4s gene fo...
            6      1  gi|226693210|dbj|AB495282.1|  Glycine max Rhg4-like gen...
            7      1  gi|226693212|dbj|AB495283.1|  Glycine max Rhg4-like gen...
            8      1  gi|389548695|gb|

In [13]:
for hit in blast_qresult:
    hit

len(blast_qresult)  # check how many hits a QueryResult has

100

In [16]:
print(f'first hit: {blast_qresult[0]}')  # retrieves first hit
print(f'last hit: {blast_qresult[-1]}')  # retrieves last hit
print(f'first three hits: {blast_qresult[:3]}')

first hit: Query: [Glycine
       max] Forrest B100B10 BAC sequence, Rhg4
  Hit: gi|1003312939|gb|KT873795.1| (93011)
       Glycine max cultivar Forrest clone BAC B100B10 genomic sequence
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0  171760.00   93011     [1042:94053]              [0:93011]
          1    1e-177     643.75     589    [86219:86798]          [84490:85076]
          2  5.3e-111     422.16     241    [40570:40811]          [39254:39494]
          3  5.3e-111     422.16     241    [40296:40536]          [39528:39769]
last hit: Query: [Glycine
       max] Forrest B100B10 BAC sequence, Rhg4
  Hit: gi|1190064283|gb|KX881928.1| (1413)
       Glycine max isolate SHMT08_L299F(F1336) serine hydroxymethyltransferas...
 HSPs: ----  --------  ---------  ------  -----

#### Hit

In [17]:
from Bio import SearchIO
blast_qresult = SearchIO.read('my_blast.xml', 'blast-xml')
blast_hit = blast_qresult[3]
print(blast_hit)

Query: [Glycine
       max] Forrest B100B10 BAC sequence, Rhg4
  Hit: gi|308083482|gb|HM367084.1| (10031)
       Glycine max cultivar Forrest leucine-rich repeat receptor-like kinase ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0   18519.30   10031    [72583:82614]              [0:10031]


#### HSP (high-scoring pair) represents region(s) in the hit sequence that contains significant alignment(s) to the query sequence. It contains the actual match between your query sequence and a database entry.

In [21]:
from Bio import SearchIO
blast_qresult = SearchIO.read('my_blast.xml', 'blast-xml')
blast_hsp = blast_qresult[0][0]
print(blast_hsp)

print(f'query_range: {blast_hsp.query_range}')
print(f'e value: {blast_hsp.evalue}')
print(f'start: {blast_hsp.hit_start}')
print(f'end: {blast_hsp.hit_end}')
print(f'query span: {blast_hsp.query_span}')
print(f'alignment length: {blast_hsp.aln_span}')
print(f'number of gaps: {blast_hsp.gap_num}')
print(f'number of identical residues: {blast_hsp.ident_num}')

      Query: [Glycine max] Forrest B100B10 BAC sequence, Rhg4
        Hit: gi|1003312939|gb|KT873795.1| Glycine max cultivar Forrest clone ...
Query range: [1042:94053] (1)
  Hit range: [0:93011] (1)
Quick stats: evalue 0; bitscore 171760.00
  Fragments: 1 (93011 columns)
     Query - TAGCCCGAACTAACATACCTCCTTTGTTAAAGCACTAAAAAACACATCCTCAATGAAAT~~~AGATG
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - TAGCCCGAACTAACATACCTCCTTTGTTAAAGCACTAAAAAACACATCCTCAATGAAAT~~~AGATG
query_range: (1042, 94053)
e value: 0.0
start: 0
end: 93011
query span: 93011
alignment length: 93011
number of gaps: 0
number of identical residues: 93011


### Searching for instances

In [26]:
# motif objects and creating motif from sequences
from Bio import motifs
from Bio.Seq import Seq
instances = [Seq('TACAA'),
                Seq('TACGC'),
                Seq('TACAC'),
                Seq('TACCC'),
                Seq('AACCC'),
                Seq('AATGC'),
                Seq('AATGC')
               ]
m = motifs.create(instances)
print(m)

TACAA
TACGC
TACAC
TACCC
AACCC
AATGC
AATGC



In [27]:
print(len(m))
print(m.consensus)

5
TACGC


In [30]:
# simplest way to search for instances is to find exact match
test_seq = Seq('TACACTGCATTACAACCCAAGCATTA')
for pos, seq in m.instances.search(test_seq):
    print('%i %s' % (pos, seq))

# we can do same thing for reverse strand (complements)
# for pos, seq in r.instances.search(test_seq):
#     print('%i %s' % (pos, seq))

0 TACAC
10 TACAA
13 AACCC


### Identifying Open Reading Frames (ORFs)

In [31]:
from Bio import SeqIO
record = SeqIO.read('NC_005816.fna', 'fasta')
table = 11
min_pro_len = 100

for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
    for frame in range(3):
        length = 3 * ((len(record)-frame) // 3)  #Multiple of three
        for pro in nuc[frame:frame+length].translate(table).split('*'):
            if len(pro) >= min_pro_len:
                print('%s...%s - length %i, strand %i, frame %i' \
                      % (pro[:30], pro[-3:], len(pro), strand, frame))

GCLMKKSSIVATIITILSGSANAASSQLIP...YRF - length 315, strand 1, frame 0
KSGELRQTPPASSTLHLRLILQRSGVMMEL...NPE - length 285, strand 1, frame 1
GLNCSFFSICNWKFIDYINRLFQIIYLCKN...YYH - length 176, strand 1, frame 1
VKKILYIKALFLCTVIKLRRFIFSVNNMKF...DLP - length 165, strand 1, frame 1
NQIQGVICSPDSGEFMVTFETVMEIKILHK...GVA - length 355, strand 1, frame 2
RRKEHVSKKRRPQKRPRRRRFFHRLRPPDE...PTR - length 128, strand 1, frame 2
TGKQNSCQMSAIWQLRQNTATKTRQNRARI...AIK - length 100, strand 1, frame 2
QGSGYAFPHASILSGIAMSHFYFLVLHAVK...CSD - length 114, strand -1, frame 0
IYSTSEHTGEQVMRTLDEVIASRSPESQTR...FHV - length 111, strand -1, frame 0
WGKLQVIGLSMWMVLFSQRFDDWLNEQEDA...ESK - length 125, strand -1, frame 1
RGIFMSDTMVVNGSGGVPAFLFSGSTLSSY...LLK - length 361, strand -1, frame 1
WDVKTVTGVLHHPFHLTFSLCPEGATQSGR...VKR - length 111, strand -1, frame 1
LSHTVTDFTDQMAQVGLCQCVNVFLDEVTG...KAA - length 107, strand -1, frame 2
RALTGLSAPGIRSQTSCDRLRELRYVPVSL...PLQ - length 119, strand -1, frame 2


In [34]:
# tracking the locations in terms of the protein counting
def find_orfs_with_trans(seq, trans_table, min_protein_length):
    answer = []
    seq_len = len(seq)
    for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
        for frame in range(3):
            trans = nuc[frame:].translate(trans_table)
            trans_len = len(trans)
            aa_start = 0
            aa_end = 0
            while aa_start < trans_len:
                aa_end = trans.find('*', aa_start)
                if aa_end == -1:
                    aa_end = trans_len  # reached the end of the protein
                if aa_end - aa_start >= min_protein_length:
                    if strand == 1:
                        start = frame + aa_start * 3
                        end = min(seq_len, frame + aa_end * 3 + 3)
                    else:
                        start = seq_len - frame - aa_end * 3 - 3
                        end = seq_len - frame - aa_start * 3
                    answer.append((start, end, strand, trans[aa_start:aa_end]))
                aa_start = aa_end + 1
    answer.sort()
    return answer

record = SeqIO.read("NC_005816.gb", "genbank")
table = 11
min_pro_len = 100
orf_list = find_orfs_with_trans(record.seq, table, min_pro_len)

for start, end, pro in orf_list:
    print('%s...%s - length %i, strand %i, %i:%i' % (pro[:30], pro[-3:], len(pro), strand, start, end))

C:\Users\sandr\anaconda3\lib\site-packages\Bio\Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


ValueError: too many values to unpack (expected 3)